In [1]:
import torch
import distributed_system as Dissys
from torch.utils.tensorboard import SummaryWriter

In [23]:
# tensorboard
tensorboard_log_path = "2-params-data_log"
tensorboard_tag_name = "total loss 3"

In [3]:
# 超参数
alpha = 0.01
num_epochs = 500

In [4]:
# 构造数据
x = [[torch.tensor([-1, -1]), torch.tensor([1, 3])],
     [torch.tensor([0, 1])],
     [torch.tensor([-0.5, 0])],
     [torch.tensor([-2, -3])]]

In [5]:
# 参数
params = torch.zeros(2, requires_grad=True)

In [24]:
# 更新参数
dissys.renew_params(params)

In [6]:
# 函数
def func(_data:torch.Tensor, _params:torch.Tensor) -> torch.Tensor:
    return torch.sum(
        (_params[0] * _data[:, 0] + _params[1] - _data[:, 1]) ** 2)


def outer_grad_func(_alpha):
    def inner_grad_func(_params_local:torch.Tensor, _params_mailbox:list[torch.Tensor]) -> torch.Tensor:
        len_mailbox = len(_params_mailbox)
        g = torch.zeros_like(_params_local.grad)
        for i in range(0, len_mailbox):
            g += _params_mailbox[i].grad
        g /= len_mailbox
        g += _params_local.grad
        g *= _alpha
        return g
    return inner_grad_func


grad_func = outer_grad_func(alpha)

In [7]:
dissys = Dissys.distributed_sys(func, x, params)

In [8]:
dissys.create_com_mat()

In [9]:
dissys.com_mat

tensor([[0, 1, 0, 1],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 1, 0]], dtype=torch.int32)

In [18]:
dissys.disconnect_nodes(0, 3)
print(dissys.com_mat)

tensor([[0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0]], dtype=torch.int32)


In [25]:
dissys.forward_all(requires_grad=False).item()

20.0

In [26]:
writer = SummaryWriter(tensorboard_log_path)
for epoch in range(num_epochs):
    dissys.backward_all()
    dissys.send()
    dissys.backward_mailbox_all()
    dissys.iterate_all(grad_func)
    # 写入tensorboard
    loss = dissys.forward_all(requires_grad=False)
    # loss = torch.log(loss).item()
    writer.add_scalar(tensorboard_tag_name, loss, epoch)
writer.close()

In [27]:
for node_i in dissys:
    print(node_i.params)

tensor([2.0037, 0.9982], requires_grad=True)
tensor([2.4029, 0.9945], requires_grad=True)
tensor([1.4447, 0.7150], requires_grad=True)
tensor([ 1.2012, -0.6006], requires_grad=True)
